# Neural Graphics Ex1: Training Your Own Diffusion Model!

## Setup environment

In [1]:
# We recommend using these utils.
# https://google.github.io/mediapy/mediapy.html
# https://einops.rocks/
!pip install mediapy einops --quiet

In [4]:
# Import essential modules. Feel free to add whatever you need.
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Seed your work
To be able to reproduce your code, please use a random seed from this point onward.

In [5]:
def seed_everything(seed):
  torch.cuda.manual_seed(seed)
  torch.manual_seed(seed)

YOUR_SEED = 180 # modify if you want
seed_everything(YOUR_SEED)

## 1. Basic Ops and UNet blocks
**Notations:**  
 * `Conv2D(kernel_size, stride, padding)` is `nn.Conv2d()`  
 * `BN` is `nn.BatchNorm2d()`  
 * `GELU` is `nn.GELU()`  
 * `ConvTranspose2D(kernel_size, stride, padding)` is `nn.ConvTranspose2d()`  
 * `AvgPool(kernel_size)` is `nn.AvgPool2d()`  
 * `Linear` is `nn.Linear()`  
 * `N`, `C`, `W` and `H` are batch size, channels num, weight and height respectively


### Basic Ops

In [ ]:
class Conv(nn.Module):
    """
    A convolutional layer that doesn’t change the image
    resolution, only the channel dimension
    Applies nn.Conv2d(3, 1, 1) followed by BN and GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the Conv layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.conv =nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(num_features=out_channels),
                                nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H, W) output tensor.
        """
        return self.conv(x)

class DownConv(nn.Module):
    """
        A convolutional layer downsamples the tensor by 2.
        The layer consists of Conv2D(3, 2, 1) followed by BN and GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the DownConv layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.downconv =nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                kernel_size=3, stride=2, padding=1),
                                nn.BatchNorm2d(num_features=out_channels),
                                nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H/2, W/2) output tensor.
        """
        return self.downconv(x)


class UpConv(nn.Module):
    """
    A convolutional layer that upsamples the tensor by 2.
    The layer consists of ConvTranspose2d(4, 2, 1) followed by
    BN and GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the UpConv layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.upconv =nn.Sequential(nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels,
                                kernel_size=4, stride=2, padding=1),
                                nn.BatchNorm2d(num_features=out_channels),
                                nn.GELU())


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H*2, W*2) output tensor.
        """
        return self.upconv(x)


class Flatten(nn.Module):
    """
    Average pooling layer that flattens a 7x7 tensor into a 1x1 tensor.
    The layer consists of AvgPool followed by GELU.
    """
    def __init__(self):
        super().__init__()
        self.flatten = nn.Sequential(nn.AvgPool2d(kernel_size=7),
                                     nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, C, 7, 7) input tensor.

        Returns:
            (N, C, 1, 1) output tensor.
        """
        return self.flatten(x)


class Unflatten(nn.Module):
    """
      Convolutional layer that unflattens/upsamples a 1x1 tensor into a
      7x7 tensor. The layer consists of ConvTranspose2D(7, 7, 0)
      followed by BN and GELU.
    """
    def __init__(self, in_channels: int):
        """
        Initializes Unflatten layer
        Args:
            in_channels (int): The number of input channels
        """
        super().__init__()
        self.unflatten = nn.Sequential(nn.ConvTranspose2d(in_channels=in_channels,out_channels=in_channels,
                                                          kernel_size=7, stride=7,padding=0),
                                       nn.BatchNorm2d(num_features=in_channels),
                                       nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, 1, 1) input tensor.

        Returns:
            (N, in_channels, 7, 7) output tensor.
        """
        return self.unflatten(x)

class FC(nn.Module):
    """
    Fully connected layer, consisting of nn.linear followed by GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the FC layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.fc =nn.Sequential(nn.Linear(in_features=in_channels, out_features=out_channels),
                               nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels) input tensor.

        Returns:
            (N, out_channels) output tensor.
        """
        return self.fc(x)

### UNet Blocks

In [9]:
class ConvBlock(nn.Module):
    """
    Two consecutive Conv operations.
    Note that it has the same input and output shape as Conv.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes ConvBlock
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.convblock = nn.Sequential(Conv(in_channels, out_channels),
                                       Conv(out_channels, out_channels))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H, W) output tensor.
        """
        return self.convblock(x)


class DownBlock(nn.Module):
    """
    DownConv followed by ConvBlock. Note that it has the same input and output
    shape as DownConv.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes DownBlock
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.downconv = nn.Sequential(DownConv(in_channels, out_channels),
                                      ConvBlock(out_channels, out_channels))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H/2, W/2) output tensor.
        """
        return self.downconv(x)


class UpBlock(nn.Module):
    """
    UpConv followed by ConvBlock.
    Note that it has the same input and output shape as UpConv
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes UpBlock
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.upconv = nn.Sequential(UpConv(in_channels, out_channels),
                                    ConvBlock(out_channels, out_channels))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H*2, W*2) output tensor.
        """
        return self.upconv(x)

class FCBlock(nn.Module):
    """
    Fully-connected Block, consisting of FC layer followed by Linear layer. Note
    that it has the same input and output shape as FC.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes FCBlock
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.fcblock = nn.Sequential(FC(in_channels,out_channels),
                                     nn.Linear(out_channels,out_channels))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels) input tensor.

        Returns:
            (N, out_channels) output tensor.
        """
        return self.fcblock(x)

## 2. Unconditional Diffusion Framework


### 2.1 UNet architecture

In [10]:
class DenoisingUNet(nn.Module):
    def __init__(
        self,
        in_channels: int,
        num_hiddens: int
    ):
        super().__init__()
        self.ConvIn = ConvBlock(in_channels, num_hiddens)
        self.DownBlock1 = DownBlock(num_hiddens, num_hiddens)
        self.DownBlock2 = DownBlock(num_hiddens, 2*num_hiddens)
        self.Flatten = Flatten()
        self.Unflatten = Unflatten(2*num_hiddens)
        self.UpBlock1 = UpBlock(4*num_hiddens, num_hiddens)
        self.UpBlock2 = UpBlock(2*num_hiddens, num_hiddens)
        self.ConvOut = ConvBlock(2*num_hiddens, num_hiddens)
        self.Conv2D = nn.Conv2d(num_hiddens, in_channels, kernel_size=(3,3), padding=(1,1))

        # time
        self.FcBlock1 = FCBlock(1, 2*num_hiddens)
        self.FcBlock2 = FCBlock(1, num_hiddens)

    def forward(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
    ) -> torch.Tensor:
        """
        Args:
            x: (N, C, H, W) input tensor.
            t: (N, 1) normalized time tensor.

        Returns:
            (N, C, H, W) output tensor.
        """
        assert x.shape[-2:] == (28, 28), "Expect input shape to be (28, 28)."

        # Down

        res_high = self.ConvIn(x)
        res_mid = self.DownBlock1(res_high)
        res_low = self.DownBlock2(res_mid)
        flattened = self.Flatten(res_low)

        # time in

        t_low = self.FcBlock1(t).unsqueeze(-1).unsqueeze(-1)
        t_mid = self.FcBlock2(t).unsqueeze(-1).unsqueeze(-1)

        # Up

        out = self.Unflatten(flattened) + t_low
        out = self.UpBlock1(torch.cat((res_low, out), dim=1)) + t_mid
        out = self.UpBlock2(torch.cat((res_mid, out), dim=1))
        out = self.Conv2D(self.ConvOut(torch.cat((res_high, out), dim=1)))

        return out

In [8]:
class Conv(nn.Module):
    """
    A convolutional layer that doesn’t change the image
    resolution, only the channel dimension
    Applies nn.Conv2d(3, 1, 1) followed by BN and GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the Conv layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.conv =nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(num_features=out_channels),
                                nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H, W) output tensor.
        """
        return self.conv(x)

class DownConv(nn.Module):
    """
        A convolutional layer downsamples the tensor by 2.
        The layer consists of Conv2D(3, 2, 1) followed by BN and GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the DownConv layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.downconv =nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                kernel_size=3, stride=2, padding=1),
                                nn.BatchNorm2d(num_features=out_channels),
                                nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H/2, W/2) output tensor.
        """
        return self.downconv(x)


class UpConv(nn.Module):
    """
    A convolutional layer that upsamples the tensor by 2.
    The layer consists of ConvTranspose2d(4, 2, 1) followed by
    BN and GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the UpConv layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.upconv =nn.Sequential(nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels,
                                kernel_size=4, stride=2, padding=1),
                                nn.BatchNorm2d(num_features=out_channels),
                                nn.GELU())


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, H, W) input tensor.

        Returns:
            (N, out_channels, H*2, W*2) output tensor.
        """
        return self.upconv(x)


class Flatten(nn.Module):
    """
    Average pooling layer that flattens a 7x7 tensor into a 1x1 tensor.
    The layer consists of AvgPool followed by GELU.
    """
    def __init__(self):
        super().__init__()
        self.flatten = nn.Sequential(nn.AvgPool2d(kernel_size=7),
                                     nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, C, 7, 7) input tensor.

        Returns:
            (N, C, 1, 1) output tensor.
        """
        return self.flatten(x)


class Unflatten(nn.Module):
    """
      Convolutional layer that unflattens/upsamples a 1x1 tensor into a
      7x7 tensor. The layer consists of ConvTranspose2D(7, 7, 0)
      followed by BN and GELU.
    """
    def __init__(self, in_channels: int):
        """
        Initializes Unflatten layer
        Args:
            in_channels (int): The number of input channels
        """
        super().__init__()
        self.unflatten = nn.Sequential(nn.ConvTranspose2d(in_channels=in_channels,out_channels=in_channels,
                                                          kernel_size=7, stride=7,padding=0),
                                       nn.BatchNorm2d(num_features=in_channels),
                                       nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels, 1, 1) input tensor.

        Returns:
            (N, in_channels, 7, 7) output tensor.
        """
        return self.unflatten(x)

class FC(nn.Module):
    """
    Fully connected layer, consisting of nn.linear followed by GELU.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the FC layer
        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels
        """
        super().__init__()
        self.fc =nn.Sequential(nn.Linear(in_features=in_channels, out_features=out_channels),
                               nn.GELU())

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, in_channels) input tensor.

        Returns:
            (N, out_channels) output tensor.
        """
        return self.fc(x)



### 2.1 DDPM Forward and Inverse Process


In [11]:
def ddpm_schedule(beta1: float, beta2: float, num_ts: int, device: str = 'cuda') -> dict:
    """Constants for DDPM training and sampling.

    Arguments:
        beta1: float, starting beta value.
        beta2: float, ending beta value.
        num_ts: int, number of timesteps.

    Returns:
        dict with keys:
            betas: linear schedule of betas from beta1 to beta2.
            alphas: 1 - betas.
            alpha_bars: cumulative product of alphas.
    """
    assert beta1 < beta2 < 1.0, "Expect beta1 < beta2 < 1.0."

    betas = torch.linspace(beta1, beta2, num_ts).to(device)
    alphas = 1 - betas
    alpha_bars = torch.cumprod(alphas, dim=0)

    out_dict = {
        'betas': betas,
        'alphas': alphas,
        'alpha_bars': alpha_bars,
    }
    return out_dict


In [12]:
def ddpm_forward(
    unet: DenoisingUNet,
    ddpm_schedule: dict,
    x_0: torch.Tensor,
    num_ts: int,
) -> torch.Tensor:
    """Algorithm 1 of the DDPM paper (not including gradient step).

    Args:
        unet: DenoisingUNet
        ddpm_schedule: dict
        x_0: (N, C, H, W) input tensor.
        num_ts: int, number of timesteps.
    Returns:
        (,) diffusion loss.
    """
    unet.train()
    # YOUR CODE HERE.
    device = x_0.device
    N = x_0.shape[0]
    ts = torch.randint(1, num_ts+1, (N,), device=device)
    epsilons = torch.randn_like(x_0, device=device)
    alpha_bars = ddpm_schedule['alpha_bars'][ts-1]
    alpha_bars = alpha_bars.view(-1, 1, 1, 1)
    xt_vec = torch.sqrt(alpha_bars)*x_0 + torch.sqrt(1-alpha_bars)*epsilons
    ts_normalized = (ts/num_ts).view(-1,1)
    epsilons_est = unet(xt_vec,ts_normalized)
    loss = nn.functional.mse_loss(epsilons, epsilons_est)

    return loss

In [13]:
@torch.inference_mode()
def ddpm_sample(
    unet: DenoisingUNet,
    ddpm_schedule: dict,
    img_wh: tuple[int, int],
    batch_size: int,
    num_ts: int
) -> torch.Tensor:
    """Algorithm 2 of the DDPM paper.

    Args:
        unet: DenoisingUNet
        ddpm_schedule: dict
        img_wh: (H, W) output image width and height.
        num_ts: int, number of timesteps.

    Returns:
        (N, C, H, W) final sample.
    """
    unet.eval()
    # YOUR CODE HERE.
    device = next(unet.parameters()).device
    H, W = img_wh
    C = unet.in_channels
    xt = torch.randn(batch_size, C, H, W, device=device)

    alpha_bars = ddpm_schedule['alpha_bars']
    beta = ddpm_schedule['beta']

    for t in range(num_ts,0,-1):
        z = torch.randn_like(xt) if t > 1 else torch.zeros_like(xt)
        at = alpha_bars[t-1].view(-1, 1, 1, 1)

        t_norm = torch.full((batch_size, 1), t / num_ts, device=device)
        epsilons_est = unet(xt,t_norm)

        x0_est = (1/torch.sqrt(at))*(xt-torch.sqrt(1-at)*epsilons_est)

        at_next = alpha_bars[t-2].view(-1, 1, 1, 1) if t > 1 else torch.ones_like(at)
        bt = beta[t-1].view(-1, 1, 1, 1)

        xt = (torch.sqrt(at_next)*bt/(1-at))*x0_est + (torch.sqrt(at)*(1-at_next)/(1-at))*xt + torch.sqrt(bt)*z

    return xt

In [14]:
# Do Not Modify
class DDPM(nn.Module):
    def __init__(
        self,
        unet: DenoisingUNet,
        betas: tuple[float, float] = (1e-4, 0.02),
        num_ts: int = 300,
        p_uncond: float = 0.1,
    ):
        super().__init__()
        self.unet = unet
        self.betas = betas
        self.num_ts = num_ts
        self.p_uncond = p_uncond
        self.ddpm_schedule = ddpm_schedule(betas[0], betas[1], num_ts)

        for k, v in ddpm_schedule(betas[0], betas[1], num_ts).items():
            self.register_buffer(k, v, persistent=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, C, H, W) input tensor.

        Returns:
            (,) diffusion loss.
        """
        return ddpm_forward(
            self.unet, self.ddpm_schedule, x, self.num_ts
        )

    @torch.inference_mode()
    def sample(
        self,
        img_wh: tuple[int, int],
        batch_size: int
    ):
        return ddpm_sample(
            self.unet, self.ddpm_schedule, img_wh, batch_size, self.num_ts
        )

### 2.3 Train your denoiser

In [15]:
import os
import shutil

def save_checkpoint(epoch, model, optimizer, scheduler, batch_losses, epoch_losses, CHECKPOINT_DIR):

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'batch_losses': batch_losses,
        'epoch_losses': epoch_losses,
    },  os.path.join(CHECKPOINT_DIR, f'ddpm_epoch_{epoch}.pth'))
    print(f"Checkpoint saved at epoch {epoch}")

def load_checkpoint(CHECKPOINT_DIR, train_from_scratch, num_hidden, lr, num_epochs, clear_folder):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Init denoiser and DDPM wrapper
    denosier_unet = DenoisingUNet(in_channels=1 , num_hiddens=num_hidden)
    ddpm = DDPM(denosier_unet, num_ts=T)

    # Optimizer and device setup - Adam optimizer with exponential learning rate decay
    optimizer = torch.optim.Adam(ddpm.parameters(), lr = lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma = 0.1**(1.0/num_epochs))
    ddpm.to(device)

    # Create/Clear folder if necessary
    if clear_folder and os.path.exists(CHECKPOINT_DIR):
        print(f"Clearing all checkpoints in: {CHECKPOINT_DIR}")
        # Delete the whole directory and recreate it
        shutil.rmtree(CHECKPOINT_DIR)
    if not os.path.exists(CHECKPOINT_DIR):
        os.makedirs(CHECKPOINT_DIR)
        print(f"Created directory: {CHECKPOINT_DIR}")

    # Logic to find the latest checkpoint and load it
    latest_checkpoint = None

    # Check for existing files in the directory
    if os.path.exists(CHECKPOINT_DIR):
        files = [f for f in os.listdir(CHECKPOINT_DIR) if f.endswith('.pth')]
        if files:
            # Sort by epoch number (assumes name format 'ddpm_epoch_X.pth')
            files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
            latest_checkpoint = os.path.join(CHECKPOINT_DIR, files[-1])

    if (not latest_checkpoint is None) and (not train_from_scratch):
        print(f"Loading checkpoint: {latest_checkpoint}")
        checkpoint = torch.load(latest_checkpoint)

        ddpm.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        start_epoch = checkpoint['epoch']
        batch_losses = checkpoint.get('batch_losses', [])
        epoch_losses = checkpoint.get('epoch_losses', [])
        print(f"Resuming from epoch {start_epoch}")

    else:
        start_epoch = 0
        batch_losses = []
        epoch_losses = []
        print(f"training from scratch")

    if start_epoch > num_epochs:
      raise IOError(f'The loaded model epoch was {start_epoch}, while the request number of epochs is {num_epochs}.')

    return start_epoch, ddpm, optimizer, scheduler, batch_losses, epoch_losses

In [16]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# eval setup
CHECKPOINT_DIR = "/content/drive/NeuralGraphicsEX1/ddpm_checkpoints"
train_from_scratch = False
clear_folder = False
eval_epochs = [1, 2, 3, 5, 10, 15, 20]

# Hyper parameters - Modify if you wish
num_hidden = 32 # 128
batch_size = 64
num_epochs = 20
lr = 1e-3
img_wh = (28, 28)
eval_batch_size=20
T=300

# initial\load checkpoint
start_epoch, ddpm, optimizer, scheduler, batch_losses, epoch_losses = load_checkpoint(CHECKPOINT_DIR, train_from_scratch, num_hidden, lr, num_epochs, clear_folder)

# Init MNIST data loaders
train_data = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_data = MNIST(root='./data', train=False, download=True, transform=ToTensor())
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(test_data, batch_size=eval_batch_size, shuffle=True) #Not usefull now, but will be for evaluating class-conditioned denoiser (3.3)

for epoch in tqdm(range(start_epoch, num_epochs)):
  ddpm.train()  # Set the model to training mode
  epoch_loss = 0.0
  for batch, (data, label) in enumerate(train_loader):
      optimizer.zero_grad()
      data = data.to(device)
      loss = ddpm(data)
      loss.backward()  # Compute gradients
      optimizer.step()  # Update weights
      batch_loss = loss.item()
      epoch_loss += batch_loss

      if batch % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch}/{len(train_loader)}], Loss: {batch_loss:.4f}")
        batch_losses.append(batch_loss)

  avg_epoch_loss = epoch_loss / len(train_loader)
  epoch_losses.append(epoch_loss)
  print(f"Epoch [{epoch+1}/{num_epochs}] - Average Loss: {avg_epoch_loss:.4f}")
  scheduler.step()

  ddpm.eval() # changes the behaior of BN and Dropouts layers
  # YOUR EVAL CODE HERE.
  with torch.no_grad():

      if (epoch + 1) in eval_epochs:
          print(f"Generating samples for epoch {epoch + 1}...")

          samples = ddpm.sample(img_wh=img_wh, batch_size=eval_batch_size)

          grid = vutils.make_grid(samples, nrow=int(eval_batch_size // 4), normalize=True)
          plt.figure(figsize=(10, 10))
          plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
          plt.title(f"Samples after {epoch + 1} epochs")
          plt.axis('off')
          plt.show()

          # Recommended: Checkpoint the model as suggested in the tips

  save_checkpoint(epoch+1, ddpm, optimizer, scheduler, batch_losses, epoch_losses, CHECKPOINT_DIR)

# Final Plotting Logic
plt.figure(figsize=(12, 5))

# Plot Batch Loss
plt.subplot(1, 2, 1)
plt.plot(batch_losses, label='Batch Loss')
plt.title('Training Batch MSE Loss')
plt.xlabel('Iterations (every 100 steps)')
plt.ylabel('MSE Loss')
plt.legend()

# Plot Epoch Loss
plt.subplot(1, 2, 2)
plt.plot(range(1, len(epoch_losses) + 1), epoch_losses, label='Epoch Loss', color='orange')
plt.title('Training Epoch Loss')
plt.xlabel('Epoch')
plt.ylabel('Total MSE Loss')
plt.legend()

plt.tight_layout()
plt.imshow(grid.permute(1, 2, 0).cpu().numpy(), cmap='gray')

AttributeError: module 'torch.nn' has no attribute 'ConvTranspose2D'

## 3. Implementing class-conditioned diffusion framework with CFG


###3.1 Adding Class-Conditioning to UNet architecture

In [ ]:
class ConditionalDenoisingUNet(nn.Module):
    def __init__(
        self,
        in_channels: int,
        num_classes: int,
        num_hiddens: int,
    ):
        super().__init__()
        # YOUR CODE HERE.

    def forward(
        self,
        x: torch.Tensor,
        c: torch.Tensor,
        t: torch.Tensor,
        mask: torch.Tensor | None = None,
    ) -> torch.Tensor:
        """
        Args:
            x: (N, C, H, W) input tensor.
            c: (N, num_classes) float condition tensor.
            t: (N, 1) normalized time tensor.
            mask: (N, 1) mask tensor. If not None, mask out condition when mask == 0.

        Returns:
            (N, C, H, W) output tensor.
        """
        assert x.shape[-2:] == (28, 28), "Expect input shape to be (28, 28)."
        # YOUR CODE HERE.
        raise NotImplementedError()

###3.2 DDPM Forward and Inverse Process with CFG

In [ ]:
def ddpm_forward(
    unet: ConditionalDenoisingUNet,
    ddpm_schedule: dict,
    x_0: torch.Tensor,
    c: torch.Tensor,
    p_uncond: float,
    num_ts: int,
) -> torch.Tensor:
    """Algorithm 3 (not including gradient step).

    Args:
        unet: ConditionalDenoisingUNet
        ddpm_schedule: dict
        x_0: (N, C, H, W) input tensor.
        c: (N,) int64 condition tensor.
        p_uncond: float, probability of unconditioning the condition.
        num_ts: int, number of timesteps.

    Returns:
        (,) diffusion loss.
    """
    unet.train()
    # YOUR CODE HERE.
    raise NotImplementedError()

In [ ]:
@torch.inference_mode()
def ddpm_cfg_sample(
    unet: ConditionalDenoisingUNet,
    ddpm_schedule: dict,
    c: torch.Tensor,
    img_wh: tuple[int, int],
    num_ts: int,
    guidance_scale: float = 5.0
) -> torch.Tensor:
    """Algorithm 4.

    Args:
        unet: ConditionalDenoisingUNet
        ddpm_schedule: dict
        c: (N,) int64 condition tensor. Only for class-conditional
        img_wh: (H, W) output image width and height.
        num_ts: int, number of timesteps.
        guidance_scale: float, CFG scale.

    Returns:
        (N, C, H, W) final sample.
    """
    unet.eval()
    # YOUR CODE HERE.
    raise NotImplementedError()

In [ ]:
# Do Not Modify
class DDPM(nn.Module):
    def __init__(
        self,
        unet: ConditionalDenoisingUNet,
        betas: tuple[float, float] = (1e-4, 0.02),
        num_ts: int = 300,
        p_uncond: float = 0.1,
    ):
        super().__init__()
        self.unet = unet
        self.betas = betas
        self.num_ts = num_ts
        self.p_uncond = p_uncond
        self.ddpm_schedule = ddpm_schedule(betas[0], betas[1], num_ts)

    def forward(self, x: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, C, H, W) input tensor.
            c: (N,) int64 condition tensor.

        Returns:
            (,) diffusion loss.
        """
        return ddpm_forward(
            self.unet, self.ddpm_schedule, x, c, self.p_uncond, self.num_ts
        )

    @torch.inference_mode()
    def sample(
        self,
        c: torch.Tensor,
        img_wh: tuple[int, int],
        guidance_scale: float = 5.0
    ):
        return ddpm_cfg_sample(
            self.unet, self.ddpm_schedule, c, img_wh, self.num_ts, guidance_scale
        )

###3.3 Train your class-conditioned denoiser

In [ ]:
# YOUR CODE HERE.

###3.4 Experiment with different guidance sacles

In [ ]:
# YOUR CODE HERE.
